In [ ]:
!pip install schedule
!pip install requests

Filename : update.py

Libraries Required

In [16]:
import os
import time
import schedule
import requests
import subprocess
import pandas as pd
from datetime import datetime

Insert the how much to cut csv for feature extraction and dataset threshold, as giving lot of URL crashes the code and 5000 is the limit of dataset.

In [10]:
MAX_ROWS_REPO = 8000
MAX_ROWS_DATASET = 5000

Downloading the online-valid.csv basically phishing repo with over 60000 phishing URLs and then trimming it to 8000 URLs for feature extraction.

In [11]:
def download_phishtank_repo():
    url = "http://data.phishtank.com/data/online-valid.csv"
    response = requests.get(url)
    with open('online-valid.csv', 'wb') as file:
        file.write(response.content)
    print("Phishtank repo downloaded.")

    # Load the phishrepo
    df_phishrepo = pd.read_csv('online-valid.csv')
    # Trim the data to the first 8000 rows
    df_phishrepo = df_phishrepo.head(MAX_ROWS_REPO)
    # Save the trimmed data
    df_phishrepo.to_csv('online-valid.csv', index=False)
    print("Phishtank repo trimmed.")

After trimming the feature_extract.py code will be executed on the csv file to extract out dataset and then further dataset will be trimmed to 5000 Rows.

In [12]:
def run_scripts():
    try:
        print("Running scripts...")
        subprocess.run(["python", "features_extract.py"])
        # Load the features dataset
        df_features = pd.read_csv('dataset_output.csv')
        # Trim the data to the first 5000 rows
        df_features = df_features.head(MAX_ROWS_DATASET)
        # Save the trimmed data
        df_features.to_csv('dataset_output.csv', index=False)
        print("Features dataset trimmed.")
        subprocess.run(["python", "rfcmodel.py"])
        print("Scripts run successfully.")
    except Exception as e:
        print("An error occurred while running the scripts:", e)

Finally setting up the update to run everyday at midnight 12:00 am to keep the phishing dataset updated.

In [ ]:
def job():
    print("Job Started at: ", datetime.now())
    download_phishtank_repo()
    run_scripts()
    print("Job Finished at: ", datetime.now())

schedule.every().day.at("00:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)